In [1]:
import sqlite3
import json

con = sqlite3.connect("git.db")

In [27]:
cur = con.cursor()

# specify the path to the JSON file
json_file_path = "events.json"

# open the file and load its contents into a Python object
with open(json_file_path, "r") as f:
    json_data = json.load(f)
## Parsing
for entry in json_data:
    ##print(entry)
    ##General Information Insertion
    user_id = entry['actor']['id']
    repo_id = entry['repo']['id']
    event_id = entry['id']
    event_type = entry['type']
    cur.execute("INSERT OR IGNORE INTO Users VALUES (?, ?, ?, ?, ?, ?)", (user_id, entry['actor']['login'], entry['actor']['display_login'], entry['actor']['gravatar_id'], entry['actor']['url'], entry['actor']['avatar_url']))
    cur.execute("INSERT OR IGNORE INTO Repo VALUES (?, ?, ?)", (repo_id, entry['repo']['name'], entry['repo']['url']))     
    con.commit()
    ##Org Insertion if needed
    try:
        org = entry['org']
        cur.execute("INSERT OR IGNORE INTO Org VALUES (?, ?, ?, ?, ?)", (org['id'], org['login'], org['gravatar_id'], org['url'], org['avatar_url']))
        con.commit()
    except KeyError:
        ##So program continues to execute
        pass
    

    ## CreateEvent Parsing
    if event_type == "CreateEvent":
        ##Event Relation Insertion
        cur.execute("INSERT INTO EventRelation VALUES (?, ?, ?, ?, ?, ?, ?)", (user_id, repo_id, event_type, event_id, org['id'], entry['public'], entry['created_at']))
        con.commit()
        
        ##Event details
        payload = entry['payload']

        cur.execute("INSERT INTO CreateEvent VALUES (?, ?, ?, ?, ?, ?)", (event_id, payload['ref'], payload['ref_type'], payload['master_branch'], payload['description'], payload['pusher_type']))
        con.commit()        

    if event_type == "DeleteEvent":
        pass
    ## PushEvent Parsing
    if event_type == "PushEvent":
        pass
    ## IssueCommentEvent Parsing
    if event_type == "IssueCommentEvent":
        pass
    ## PullRequestingEvent
    if event_type == "PullRequestEvent":
        pass

hi


In [31]:
## Searching and Examples of new uses
cur.execute("SELECT * FROM EventRelation")
print(cur.fetchall())

[('49699333', '484395347', 'CreateEvent', '28179866030', '66235900', 1, '2023-04-04T00:59:23Z')]
